In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cd /content/drive/MyDrive/hackathon; git clone --depth 1 https://github.com/google/automl

In [3]:
!cd /content/drive/MyDrive/hackathon/automl/efficientdet; pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-vbzqc26j
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-vbzqc26j
  Using cached tensorflow-2.9.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.7 MB)
     |████████████████████████████████| 5.8 MB 15.5 MB/s 
     |████████████████████████████████| 1.6 MB 58.6 MB/s 
     |████████████████████████████████| 438 kB 70.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installatio

In [4]:
!cd /content/drive/MyDrive/hackathon/automl/efficientdet; pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 665.5 MB 227 kB/s
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)
  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.1
    Uninstalling tensorflow-2.9.1:
      Successfully uninstalled tensorflow-2.9.1


In [5]:
!cd /content/drive/MyDrive/hackathon/automl/efficientdet; pip install tf-models-official==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!cd /content/drive/MyDrive/hackathon/automl/efficientdet; pip install tensorflow_io==0.23.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# !pip install wandb
# !wandb login

In [8]:
'''
import wandb
import tensorflow as tf

wandb.init(project="my-test-project", entity="yj_synstealer")
'''

'\nimport wandb\nimport tensorflow as tf\n\nwandb.init(project="my-test-project", entity="yj_synstealer")\n'

In [9]:
import os
import sys
import tensorflow.compat.v1 as tf

sys.path.append('/content/drive/MyDrive/hackathon/automl/efficientdet')

import hparams_config
from tf2 import anchors
from model_inspect import ModelInspector

In [10]:
MODEL = 'efficientdet-d2' 

def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz
    !tar zxf {m}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model in {}'.format(ckpt_path))

Use model in /content/efficientdet-d2


In [11]:
class TRAIN_CFG:
  model_name = 'efficientdet-d2' # efficientdet 모델명
  strategy = '' # tpu, 여러개의 GPU들, 단일 GPU 일때 학습 strategy 설정. 
  model_dir = '/content/drive/MyDrive/hackathon/model_trained_KV' # 학습된 모델이 저장될 위치
  pretrained_ckpt = '/content/drive/MyDrive/efficientdet-d2' # coco로 pretrained된 checkpoint 파일이 있는 디렉토리 위치
  # num_classes는 8개임. 
  hparams = 'num_classes=8,moving_average_decay=0,mixed_precision=true'
  use_xla = False
  use_fake_data = False
  # max_instances_per_image를 200으로 설정하고, D2 모델이 상대적으로 더 크기 때문에 batch_size를 8로 설정시 GPU Out of Memory 발생 
  batch_size = 4
  eval_samples = 500 # evaluation image 데이터 갯수
  steps_per_execution = 1 # train 시 steps 횟수
  num_examples_per_epoch = 2000 # 1 epochs 시 적용하는 examples 개수 
  num_epochs = 100 # epochs 횟수
  train_file_pattern = '/content/drive/MyDrive/tfrecord/train/KV/*.tfrecords' # 학습용 tfrecords를 glob 형태로 가져오는 표현식. 
  val_file_pattern = '/content/drive/MyDrive/tfrecord/val/*.tfrecords' # 검증용 tfrecords를 glob 형태로 가져오는 표현식. 
  val_json_file = None # optional coco validation json 
  mode = 'traineval' # train만 적용 또는 train과 eval함께 적용(traineval)
  num_shards = 100 # 
  # max_instances_per_image는 기본 100개임. 1개의 이미지에  특정 object의 갯수가 100개를 초과하는 경우가 있음. 200으로 수정. 
  max_instances_per_image = 200
  
  num_cores = 2 # tpu 일때 8 적용.  
  tpu = None
  gcp_project = None
  tpu_zone = None
  eval_master = ''
  eval_name = None
  tf_random_seed = 2021
  profile = False
  debug = False

In [12]:
from tf2.train import setup_model
import hparams_config

import utils
from tf2 import tfmot
from tf2 import train_lib
from tf2 import util_keras

config = hparams_config.get_detection_config(TRAIN_CFG.model_name)
config.override(TRAIN_CFG.hparams)

steps_per_epoch = TRAIN_CFG.num_examples_per_epoch // TRAIN_CFG.batch_size

if tf.config.list_physical_devices('GPU'):
  ds_strategy = tf.distribute.OneDeviceStrategy('device:GPU:0')
else:
  ds_strategy = tf.distribute.OneDeviceStrategy('device:CPU:0')

print(ds_strategy)

params = dict(
      profile=TRAIN_CFG.profile,
      mode = TRAIN_CFG.mode,
      model_name=TRAIN_CFG.model_name,
      steps_per_execution=TRAIN_CFG.steps_per_execution,
      num_epochs = TRAIN_CFG.num_epochs,
      model_dir=TRAIN_CFG.model_dir,
      steps_per_epoch=steps_per_epoch,
      strategy=TRAIN_CFG.strategy,
      batch_size=TRAIN_CFG.batch_size,
      tf_random_seed=TRAIN_CFG.tf_random_seed,
      debug=TRAIN_CFG.debug,
      val_json_file=TRAIN_CFG.val_json_file,
      eval_samples=TRAIN_CFG.eval_samples,
      num_shards=ds_strategy.num_replicas_in_sync,
      # max_instances_per_image는 기본 100개임. 1개의 이미지에  특정 object의 갯수가 100개를 초과하는 경우가 있음. 200으로 수정.
      max_instances_per_image = TRAIN_CFG.max_instances_per_image
      )

config.override(params, True)

# image size를 tuple 형태로 변환. 512는 (512, 512)로 '1920x880' 은 (1920, 880) 으로 변환.  
config.image_size = utils.parse_image_size(config.image_size)
print(config)

act_type: swish
alpha: 0.25
anchor_scale: 4.0
apply_bn_for_resampling: true
aspect_ratios:
- 1.0
- 2.0
- 0.5
autoaugment_policy: null
backbone_config: null
backbone_name: efficientnet-b2
batch_size: 4
box_class_repeats: 3
box_loss_weight: 50.0
ckpt_var_scope: null
clip_gradients_norm: 10.0
conv_after_downsample: false
conv_bn_act_pattern: false
data_format: channels_last
dataset_type: null
debug: false
delta: 0.1
drop_remainder: true
eval_samples: 500
first_lr_drop_epoch: 200.0
fpn_cell_repeats: 5
fpn_config: null
fpn_name: null
fpn_num_filters: 112
fpn_weight_method: null
gamma: 1.5
grad_checkpoint: false
grid_mask: false
heads:
- object_detection
image_size: !!python/tuple
- 768
- 768
img_summary_steps: null
input_rand_hflip: true
iou_loss_type: null
iou_loss_weight: 1.0
is_training_bn: true
jitter_max: 2.0
jitter_min: 0.1
label_map: null
label_smoothing: 0.0
learning_rate: 0.08
loss_scale: null
lr_decay_method: cosine
lr_warmup_epoch: 1.0
lr_warmup_init: 0.008
map_freq: 5
max_instan

In [13]:
import utils
import tensorflow as tf
from tf2 import tfmot
from tf2 import train_lib
from tf2 import util_keras
# P100에서는 적용할 필요 없음. 
'''
precision = utils.get_precision(config.strategy, config.mixed_precision)
policy = tf.keras.mixed_precision.Policy(precision)
tf.keras.mixed_precision.set_global_policy(policy)
'''

'\nprecision = utils.get_precision(config.strategy, config.mixed_precision)\npolicy = tf.keras.mixed_precision.Policy(precision)\ntf.keras.mixed_precision.set_global_policy(policy)\n'

In [14]:
import dataloader

def get_dataset(is_training, config):
  # is_training이 True이면 TRAIN_CFG의 train_file_pattern, 그렇지 아니면 val_file_pattern
  file_pattern = (
    TRAIN_CFG.train_file_pattern
    if is_training else TRAIN_CFG.val_file_pattern)
  if not file_pattern:
    raise ValueError('No matching files.')

  return dataloader.InputReader(
    file_pattern,
    is_training=is_training,
    use_fake_data=TRAIN_CFG.use_fake_data,
    max_instances_per_image=config.max_instances_per_image,
    debug=TRAIN_CFG.debug)(
        config.as_dict())

In [15]:
from tf2 import train_lib
from tf2 import train

def get_efficientdet_model(config):
  # 8개의 class를 가진 efficientdet d2 모델을 생성. 
  model = train_lib.EfficientDetNetTrain(config=config)
  model = train.setup_model(model, config)
  # 만약 pretrained 모델이 있으면, 해당 checkpoint weight를 모델로 로딩. 이때 classification layer는 제외. 
  #class TRAIN_CFG: pretrained_ckpt = '/content/efficientdet-d2' 
  if TRAIN_CFG.pretrained_ckpt:
    ckpt_path = tf.train.latest_checkpoint(TRAIN_CFG.pretrained_ckpt)
    # classfication layer를 제외하고 pretrained weight를 생성된 모델로 로딩. 
    util_keras.restore_ckpt(
        model,
        ckpt_path,
        config.moving_average_decay,
        exclude_layers=['class_net'])
    
  train.init_experimental(config)
  return model

model = get_efficientdet_model(config)
model.summary()

/content/drive/MyDrive/hackathon/automl/efficientdet/utils.py:23: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  from tensorflow.python.tpu import tpu_function  # pylint:disable=g-direct-tensorflow-import
/content/drive/MyDrive/hackathon/automl/efficientdet/utils.py:255: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  for u in self.updates:


Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b2 (Model)     multiple                  7267314   
                                                                 
 resample_p6 (ResampleFeatur  multiple                 39984     
 eMap)                                                           
                                                                 
 resample_p7 (ResampleFeatur  multiple                 0         
 eMap)                                                           
                                                                 
 fpn_cells (FPNCells)        multiple                  678479    
                                                                 
 class_net (ClassNet)        multiple                  56856     
                                                                 
 box_net (BoxNet)            multiple                  52788     


In [16]:
print(config.mode)

traineval


In [ ]:
from tf2 import train
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import datetime

config.batch_size = 4
tr_steps_per_epoch = 3000//config.batch_size
val_steps_per_epoch = 500//config.batch_size
print('tr_steps_per_epoch:', tr_steps_per_epoch, 'val_steps_per_epoch:', val_steps_per_epoch)

log_dir = "/content/drive/MyDrive/hackathon/logs/KV/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
val_dataset = get_dataset(False, config) if 'eval' in config.mode else None
model.fit(
    get_dataset(True, config),
    epochs=100,
    steps_per_epoch=tr_steps_per_epoch,
    callbacks=[train_lib.get_callbacks(config.as_dict(), val_dataset), tensorboard_callback],
    validation_data=val_dataset,
    validation_steps=val_steps_per_epoch)

tf.keras.backend.clear_session()

tr_steps_per_epoch: 750 val_steps_per_epoch: 125
Epoch 1/100


/content/drive/MyDrive/hackathon/automl/efficientdet/utils.py:23: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  from tensorflow.python.tpu import tpu_function  # pylint:disable=g-direct-tensorflow-import
/content/drive/MyDrive/hackathon/automl/efficientdet/utils.py:255: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  for u in self.updates:


750/750 [==============================] - ETA: 0s - det_loss: 11.8044 - cls_loss: 11.6153 - box_loss: 0.0038 - reg_l2_loss: 0.0959 - loss: 11.9004 - learning_rate: 0.0060 - gradient_norm: 3.1159
Epoch 00001: saving model to /content/drive/MyDrive/hackathon/model_trained_KV/ckpt-1
750/750 [==============================] - 776s 946ms/step - det_loss: 11.7893 - cls_loss: 11.6003 - box_loss: 0.0038 - reg_l2_loss: 0.0959 - loss: 11.8853 - learning_rate: 0.0060 - gradient_norm: 3.1149 - val_det_loss: 0.5651 - val_cls_loss: 0.3618 - val_box_loss: 0.0041 - val_reg_l2_loss: 0.0960 - val_loss: 0.6611
Epoch 2/100
750/750 [==============================] - ETA: 0s - det_loss: 0.4325 - cls_loss: 0.2756 - box_loss: 0.0031 - reg_l2_loss: 0.0960 - loss: 0.5285 - learning_rate: 0.0050 - gradient_norm: 2.7211
Epoch 00002: saving model to /content/drive/MyDrive/hackathon/model_trained_KV/ckpt-2
750/750 [==============================] - 705s 940ms/step - det_loss: 0.4323 - cls_loss: 0.2755 - box_loss: 